In [1]:
import pandas as pd
import pyspark
import itertools
frac = 0.05

In [2]:
postulantes = pd.read_csv('Sources/postulantes_proc.csv').sample(frac=frac)
avisos = pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv').sample(frac=frac)
postulaciones = pd.read_csv('Sources/fiuba_4_postulaciones_concat.csv')

In [3]:
test = []
for x in itertools.product(avisos['idaviso'],postulantes['idpostulante']):
    test.append(x)

In [4]:
test = pd.DataFrame.from_records(test)

In [5]:
test.rename({0:'idaviso',1:'idpostulante'}, axis=1, inplace=True)

In [6]:
postulaciones.drop(['Unnamed: 0','fechapostulacion'], axis=1, inplace=True)

In [7]:
postulaciones['sepostulo'] = 1

In [ ]:
postulaciones.shape

In [8]:
postulaciones = pd.concat([postulaciones,test])

In [ ]:
postulaciones.shape

In [9]:
postulaciones = postulaciones.drop_duplicates(['idaviso','idpostulante'])
postulaciones['sepostulo'].fillna(0,inplace=True)

In [ ]:
postulaciones.shape

In [10]:
postulaciones.to_csv('Sources/postulaciones_for_training.csv')